In [1]:
import pandas as pd
import numpy as np

In [2]:
y_train = pd.read_csv('data/AMF_train_Y.csv')

In [33]:
from helper import read_x_train, fill_nan
x_train = read_x_train('data/AMF_train_X.csv', includeShare=True, includeDay=True)
x_train = fill_nan(x_train)

In [34]:
y_train

,Trader,type
0,Trader_285,MIX
1,Trader_114,NON HFT
2,Trader_110,NON HFT
3,Trader_57,NON HFT
4,Trader_128,NON HFT
...,...,...
81,Trader_140,MIX
82,Trader_451,MIX
83,Trader_438,MIX
84,Trader_278,HFT


In [35]:
n_rows, n_cols = x_train.shape

### The NaN ratio in each columns

In [36]:
x_train.isnull().sum()/n_rows

Share                         0.0
Day                           0.0
Trader                        0.0
OTR                           0.0
OCR                           0.0
OMR                           0.0
min_time_two_events           0.0
mean_time_two_events          0.0
10_p_time_two_events          0.0
med_time_two_events           0.0
25_p_time_two_events          0.0
75_p_time_two_events          0.0
90_p_time_two_events          0.0
max_time_two_events           0.0
min_lifetime_cancel           0.0
mean_lifetime_cancel          0.0
10_p_lifetime_cancel          0.0
med_lifetime_cancel           0.0
25_p_lifetime_cancel          0.0
75_p_lifetime_cancel          0.0
90_p_lifetime_cancel          0.0
max_lifetime_cancel           0.0
NbTradeVenueMic               0.0
MaxNbTradesBySecond           0.0
MeanNbTradesBySecond          0.0
min_dt_TV1                    0.0
mean_dt_TV1                   0.0
med_dt_TV1                    0.0
min_dt_TV1_TV2                0.0
mean_dt_TV1_TV

In [40]:
x_train

,Share,Day,Trader,OTR,OCR,OMR,min_time_two_events,mean_time_two_events,10_p_time_two_events,med_time_two_events,...,min_dt_TV1_TV2,mean_dt_TV1_TV2,med_dt_TV1_TV2,min_dt_TV1_TV3,mean_dt_TV1_TV3,med_dt_TV1_TV3,min_dt_TV1_TV4,mean_dt_TV1_TV4,med_dt_TV1_TV4,NbSecondWithAtLeatOneTrade
0,8,24,Trader_10,2.272727,8.333333,12.500000,0.0,5117.830300,0.000000,419.688500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
1,87,29,Trader_10,1.696629,25.166667,21.571429,0.0,1846.968401,0.000074,0.003374,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15
2,87,23,Trader_10,1.482759,47.300000,118.250000,0.0,686.300630,0.000071,0.000599,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63
3,12,11,Trader_10,1.705882,14.500000,29.000000,0.0,2174.335265,0.000000,6.152666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,87,9,Trader_10,1.517730,26.750000,0.000000,0.0,944.008551,0.000071,0.001364,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105777,84,27,Trader_60,1.150000,23.000000,0.000000,0.0,736.500080,0.000000,0.000188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
105778,102,18,Trader_60,1.352941,23.000000,0.000000,0.0,63.672695,0.000000,0.000963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
105779,12,13,Trader_60,1.146667,86.000000,0.000000,0.0,708.440816,0.000000,0.000578,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16
105780,27,30,Trader_60,1.179191,204.000000,0.000000,0.0,384.893976,0.000000,0.000536,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35


In [41]:
x_train_data = np.array(x_train.drop(['Trader'], axis = 1))

In [42]:
ind2type = {0: 'HFT', 1: 'MIX', 2: 'NON HFT'}
type2ind = {'HFT': 0, 'MIX': 1, 'NON HFT': 2}

In [43]:
y_train

,Trader,type
0,Trader_285,MIX
1,Trader_114,NON HFT
2,Trader_110,NON HFT
3,Trader_57,NON HFT
4,Trader_128,NON HFT
...,...,...
81,Trader_140,MIX
82,Trader_451,MIX
83,Trader_438,MIX
84,Trader_278,HFT


In [44]:
name2type = dict(zip(y_train.Trader, y_train.type))

In [45]:
x_train = x_train.fillna(-1)

In [46]:
x_train['label'] = x_train.apply(lambda row: 0 if name2type[row.Trader] == 'HFT' else (1 if name2type[row.Trader] == 'MIX' else 2), axis=1)

In [47]:
y_train_data = x_train['label']

In [48]:
from sklearn.model_selection import train_test_split

In [51]:
x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(x_train_data, y_train_data, test_size=0.3)

In [52]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(x_train_b, y_train_b)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [50]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC

# Create the SVM
svm = LinearSVC(random_state=42)

# Make it an Multilabel classifier

# Fit the data to the Multilabel classifier
svm.fit(x_train_b, y_train_b)

# Get predictions for test data
y_test_pred = svm.predict(x_test_b)

In [20]:
len(np.where(y_test_pred == y_test_b)[0]) / len(y_test_b)

0.47981723648968017

In [23]:
x_real_test = read_x_train('data/AMF_test_X.csv', includeShare=False, includeDay=False)
x_real_test = fill_nan(x_real_test)

In [24]:
x_real_test_data = np.array(x_real_test)[:,1:]

In [25]:
y_test_predict = svm.predict(x_real_test_data)

In [26]:
x_real_test_data.shape

(85304, 35)

In [27]:
x_train_data.shape

(105782, 35)

In [28]:
y_test_predict


array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [29]:
res = [(row.Trader, y_test_predict[i])  for i, row in x_real_test.iterrows()]
    

In [30]:
dicts = {}
for row in res:
    if row[0] in dicts:
        dicts[row[0]].append(row[1])
    else:
        dicts[row[0]] = [row[1]]

In [31]:
final = {}
for k,v in dicts.items():
    if len(np.where(np.array(v) == 0)[0])/len(v) > 0.4:
        final[k] = 'HFT'
    elif len(np.where(np.array(v) == 0)[0])/len(v) > 0.2:
        final[k] = 'MIX'
    else:
        final[k] = 'NON HFT'

In [32]:
import csv
(pd.DataFrame.from_dict(data=final, orient='index').to_csv('dict_file.csv'))

In [162]:
np.where(dicts['Adelaide']==0)[0]

array([], dtype=int64)

In [161]:
dicts['Adelaide']

[2, 2, 2, 2, 1, 0, 0, 0, 2, 1]